## Usage

* `MODEL_ID`: The ID of the model to quantize (e.g., `mlabonne/EvolCodeLlama-7b`).
* `QUANTIZATION_METHOD`: The quantization method to use.

## Quantization methods

* `q2_k`: Uses Q4_K for the attention.vw and feed_forward.w2 tensors, Q2_K for the other tensors.
* `q3_k_l`: Uses Q5_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K
* `q3_k_m`: Uses Q4_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K
* `q3_k_s`: Uses Q3_K for all tensors
* `q4_0`: Original quant method, 4-bit.
* `q4_1`: Higher accuracy than q4_0 but not as high as q5_0. However has quicker inference than q5 models.
* `q4_k_m`: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K
* `q4_k_s`: Uses Q4_K for all tensors
* `q5_0`: Higher accuracy, higher resource usage and slower inference.
* `q5_1`: Even higher accuracy, resource usage and slower inference.
* `q5_k_m`: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K
* `q5_k_s`:  Uses Q5_K for all tensors
* `q6_k`: Uses Q8_K for all tensors
* `q8_0`: Almost indistinguishable from float16. High resource use and slow. Not recommended for most users.

As a rule of thumb, **I recommend using Q5_K_M** as it preserves most of the model's performance. Alternatively, you can use Q4_K_M if you want to save some memory. In general, K_M versions are better than K_S versions. I cannot recommend Q2_K or Q3_* versions, as they drastically decrease model performance. T4 GPUs (Free on Colab) can handle Q5_K_M models, up to Q6_K.

In [ ]:
#@title Install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
!pip install -r llama.cpp/requirements.txt

In [ ]:
# Variables
MODEL_ID = "R136a1/Lengkuas"
QUANTIZATION_METHODS = ["q4_k_m", "q5_k_m"]

# Constants
MODEL_NAME = MODEL_ID.split('/')[-1]

# Download the model
%cd /content/
!git clone https://github.com/TheBlokeAI/AIScripts.git
!cd AIScripts ;python hub_download.py --symlinks false $MODEL_ID "/content/"$MODEL_NAME

# Convert to fp16
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.fp16.bin"
!python llama.cpp/convert.py {MODEL_NAME} --outtype f16 --outfile {fp16}

# Quantize the model for each method in the QUANTIZATION_METHODS list
for method in QUANTIZATION_METHODS:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{method.upper()}.gguf"
    !./llama.cpp/quantize {fp16} {qtype} {method}

In [ ]:
#@title test inference
# !wget https://huggingface.co/datasets/royallab/PIPPA-cleaned/resolve/main/pippa_raw_fix.parquet
# !./llama.cpp/perplexity -ngl 100 -m /content/Lengkuas/lengkuas.Q5_K_M.gguf -f /content/pippa_raw_fix.parquet
!./llama.cpp/main -ngl 100 -m /content/Lengkuas/lengkuas.Q5_K_M.gguf --prompt "Once upon a time"

In [ ]:
#@Upload to HuggingFace
!cd AIScripts ;python hub_upload.py --branch gguf "R136a1/Madang" "/content/Lengkuas/lengkuas.Q6_K.gguf"